This notebook creates an RNN for sentence classification. It can be ran by changing the runtime type to "GPU" and selecting "run all".

In [ ]:
!pip install -U torchtext==0.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
from torchtext import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenizer_language = 'en_core_web_sm')
LABEL = data.LabelField(dtype = torch.float)

In [ ]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


# Train/Validation Splits
In the below cell, we can try different training and validation set splits. We test for splits of 0.2, 0.5, and 0.7 (the default).

In [ ]:
import random

# Here we can try different train/valid splits
# default is 0.7
train_data, valid_data = train_data.split(random_state = random.seed(SEED))

# train_data, valid_data = train_data.split(split_ratio = 0.2, random_state = random.seed(SEED))

# train_data, valid_data = train_data.split(split_ratio = 0.5, random_state = random.seed(SEED))

print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


Here, we need to build a vocabulary, and we are specifying that the maximum size is 25,000.

In [ ]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


# Batch Sizes
In the below cell, we test different batch sizes of 32, 64, and 128 for the iterator.

In [ ]:
# We can adjust the batch size here
BATCH_SIZE = 64
# BATCH_SIZE = 32
# BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device)

In [ ]:
import torch.nn as nn

class RNN(nn.Module):
  def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
    super().__init__()
    self.embedding = nn.Embedding(input_dim, embedding_dim)
    self.rnn = nn.RNN(embedding_dim, hidden_dim)
    self.fc = nn.Linear(hidden_dim, output_dim)

  def forward(self, text):
    embedded = self.embedding(text)
    output, hidden = self.rnn(embedded)

    assert torch.equal(output[-1,:,:], hidden.squeeze(0))
    return self.fc(hidden.squeeze(0))

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

def count_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,592,105 trainable parameters


# Training and Choosing Optimizer
In the below cell, we expirment with two different optimizers: SGD and Adam

In [ ]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)
# optimizer = optim.Adam(model.parameters())

criterion = nn.BCEWithLogitsLoss()

# put the model and criterion on the GPU (if we have access to on)
model = model.to(device)
criterion = criterion.to(device)

In the following cells, we define a number of functions for obtaining performance metrics.

In [ ]:
def binary_accuracy(preds, y):
  rounded_preds = torch.round(torch.sigmoid(preds))
  correct = (rounded_preds == y).float()
  acc = correct.sum() / len(correct)
  return acc

In [ ]:
def getStats(preds, y):

  rounded_preds = torch.round(torch.sigmoid(preds))

  preds_list = rounded_preds.tolist()
  labels = y.tolist()

  tp = 0.0
  tn = 0.0
  fp = 0.0
  fn = 0.0

  for i in range(len(preds_list)):
    if (preds_list[i] == 1 and labels[i] == 1): tp += 1
    if (preds_list[i] == 0 and labels[i] == 0): tn += 1
    if (preds_list[i] == 1 and labels[i] == 0): fp += 1
    if (preds_list[i] == 0 and labels[i] == 1): fn += 1

  return tp, tn, fp, fn

In [ ]:
def binary_precision(tp, fp):
  if (tp + fp == 0.0): return 0.0
  return tp/(tp + fp)

In [ ]:
def binary_recall(tp, fn):
  if (tp + fn == 0.0): return 0.0
  return tp/(tp + fn)

In [ ]:
def binary_f1(precision, recall):
  if (precision + recall == 0.0): return 0.0
  return (2 * precision * recall) / (precision + recall)

In [ ]:
def train(model, iterator, optimizer, criterion):
  epoch_loss = 0
  epoch_acc = 0
  epoch_prec = 0
  epoch_recall = 0
  epoch_f1 = 0

  model.train()

  for batch in iterator:
    optimizer.zero_grad()
    predictions = model(batch.text).squeeze(1)
    loss = criterion(predictions, batch.label)
    acc = binary_accuracy(predictions, batch.label)

    tp, tn, fp, fn = getStats(predictions, batch.label)
    prec = binary_precision(tp, fp)
    recall = binary_recall(tp, fn)
    f1_score = binary_f1(prec, recall)

    loss.backward()

    optimizer.step()

    epoch_loss += loss.item()
    epoch_acc += acc.item()
    epoch_prec += prec
    epoch_recall += recall
    epoch_f1 += f1_score

  return epoch_loss / len(iterator), epoch_acc / len(iterator), epoch_prec / len(iterator), epoch_recall / len(iterator), epoch_f1 / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
  epoch_loss = 0
  epoch_acc = 0
  epoch_prec = 0
  epoch_recall = 0
  epoch_f1 = 0
  
  model.eval()

  with torch.no_grad():
    for batch in iterator:
      predictions = model(batch.text).squeeze(1)
      loss = criterion(predictions, batch.label)
      acc = binary_accuracy(predictions, batch.label)

      tp, tn, fp, fn = getStats(predictions, batch.label)
      prec = binary_precision(tp, fp)
      recall = binary_recall(tp, fn)
      f1_score = binary_f1(prec, recall)

      epoch_loss += loss.item()
      epoch_acc += acc.item()
      epoch_prec += prec
      epoch_recall += recall
      epoch_f1 += f1_score

  return epoch_loss / len(iterator), epoch_acc / len(iterator), epoch_prec / len(iterator), epoch_recall / len(iterator), epoch_f1 / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
  elapsed_time = end_time - start_time
  elapsed_mins = int(elapsed_time / 60)
  elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
  return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
  start_time = time.time()

  train_loss, train_acc, train_prec, train_recall, train_f1 = train(model, train_iterator, optimizer, criterion)
  valid_loss, valid_acc, valid_prec, valid_recall, valid_f1 = evaluate(model, valid_iterator, criterion)

  end_time = time.time()

  epoch_mins, epoch_secs = epoch_time(start_time, end_time)

  if valid_loss < best_valid_loss:
    best_valid_loss = valid_loss
    torch.save(model.state_dict(), 'tut1-model.pt')

  print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
  print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Train Prec: {train_prec*100:.2f}% | Test Recall: {train_recall*100:.2f}% | Test F1 Score: {train_f1*100:.2f}%')
  print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% | Valid Prec: {valid_prec*100:.2f}% | Valid Recall: {valid_recall*100:.2f}% | Valid F1 Score: {valid_f1*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 10s
	Train Loss: 0.694 | Train Acc: 50.28% | Train Prec: 42.42% | Test Recall: 39.65% | Test F1 Score: 27.26%
	 Val. Loss: 0.697 |  Val. Acc: 50.17% | Valid Prec: 50.57% | Valid Recall: 75.54% | Valid F1 Score: 57.60%
Epoch: 02 | Epoch Time: 0m 8s
	Train Loss: 0.693 | Train Acc: 49.63% | Train Prec: 39.03% | Test Recall: 21.99% | Test F1 Score: 15.49%
	 Val. Loss: 0.697 |  Val. Acc: 51.20% | Valid Prec: 51.16% | Valid Recall: 91.16% | Valid F1 Score: 65.15%
Epoch: 03 | Epoch Time: 0m 8s
	Train Loss: 0.693 | Train Acc: 50.03% | Train Prec: 42.10% | Test Recall: 22.83% | Test F1 Score: 16.29%
	 Val. Loss: 0.697 |  Val. Acc: 51.27% | Valid Prec: 51.20% | Valid Recall: 91.29% | Valid F1 Score: 65.21%
Epoch: 04 | Epoch Time: 0m 8s
	Train Loss: 0.693 | Train Acc: 49.90% | Train Prec: 42.21% | Test Recall: 23.17% | Test F1 Score: 16.48%
	 Val. Loss: 0.697 |  Val. Acc: 49.70% | Valid Prec: 50.44% | Valid Recall: 66.81% | Valid F1 Score: 53.40%
Epoch: 05 | Epoch Time:

In [ ]:
model.load_state_dict(torch.load('tut1-model.pt'))
test_loss, test_acc, test_prec, test_recall, test_f1 = evaluate(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% | Test Prec: {test_prec*100:.2f}% | Test Recall: {test_recall*100:.2f}% | Test F1 Score: {test_f1*100:.2f}%')

Test Loss: 0.710 | Test Acc: 45.08% | Test Prec: 47.95% | Test Recall: 69.95% | Test F1 Score: 52.87%
